In [25]:
from os.path import join as pjoin

from common.skeleton import Skeleton
import numpy as np
import os
from common.quaternion import *
from paramUtil import *

import torch
from tqdm import tqdm
import os
import rotation_conversions as geometry

In [26]:
l_idx1, l_idx2 = 5, 8
# Right/Left foot
fid_r, fid_l = [8, 11], [7, 10]
# Face direction, r_hip, l_hip, sdr_r, sdr_l
face_joint_indx = [2, 1, 17, 16]
# l_hip, r_hip
r_hip, l_hip = 2, 1
joints_num = 22
n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
kinematic_chain = t2m_kinematic_chain
def process_file(positions, angle, feet_thre):

    """ Get Foot Contacts """
    global_positions = positions.copy()

    def foot_detect(positions, thres):
        velfactor, heightfactor = np.array([thres, thres]), np.array([3.0, 2.0])

        feet_l_x = (positions[1:, fid_l, 0] - positions[:-1, fid_l, 0]) ** 2
        feet_l_y = (positions[1:, fid_l, 1] - positions[:-1, fid_l, 1]) ** 2
        feet_l_z = (positions[1:, fid_l, 2] - positions[:-1, fid_l, 2]) ** 2
        #     feet_l_h = positions[:-1,fid_l,1]
        #     feet_l = (((feet_l_x + feet_l_y + feet_l_z) < velfactor) & (feet_l_h < heightfactor)).astype(np.float)
        feet_l = ((feet_l_x + feet_l_y + feet_l_z) < velfactor).astype(np.float32)

        feet_r_x = (positions[1:, fid_r, 0] - positions[:-1, fid_r, 0]) ** 2
        feet_r_y = (positions[1:, fid_r, 1] - positions[:-1, fid_r, 1]) ** 2
        feet_r_z = (positions[1:, fid_r, 2] - positions[:-1, fid_r, 2]) ** 2
        #     feet_r_h = positions[:-1,fid_r,1]
        #     feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor) & (feet_r_h < heightfactor)).astype(np.float)
        feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor)).astype(np.float32)
        return feet_l, feet_r
    #
    feet_l, feet_r = foot_detect(positions, feet_thre)

    def get_cont6d_params(positions, angle):
        cont_6d_params = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(torch.tensor(angle))).numpy()
        # skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
        # (seq_len, joints_num, 4)
        # quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=True)
        # (seq_len, 4)
        # r_rot = quat_params[:, 0].copy()
        r_rot=geometry.axis_angle_to_quaternion(torch.tensor(angle))[:,0].numpy()
        
        velocity = (positions[1:, 0] - positions[:-1, 0]).copy()
        velocity = qrot_np(r_rot[1:], velocity)
        r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
        return cont_6d_params, r_velocity, velocity, r_rot
    cont_6d_params, r_velocity, velocity, r_rot = get_cont6d_params(positions, angle)

    def get_rifke(positions):
        '''Local pose'''
        positions[:,1:, 0] -= positions[:, 0:1, 0]
        positions[:,1:, 2] -= positions[:, 0:1, 2]
        return positions

    '''Root height'''
    positions = get_rifke(positions)
    root_y = positions[:, 0, 1:2]

    root_data = np.concatenate([positions[:-1,0], root_y[:-1]], axis=-1)

    '''Get Joint Rotation Representation'''
    # (seq_len, (joints_num-1) *6) quaternion for skeleton joints
    rot_data = cont_6d_params[:, 1:].reshape(len(cont_6d_params), -1)

    '''Get Joint Rotation Invariant Position Represention'''
    # (seq_len, (joints_num-1)*3) local joint position
    ric_data = positions[:, 1:].reshape(len(positions), -1)

    '''Get Joint Velocity Representation'''
    # (seq_len-1, joints_num*3)
    local_vel = qrot_np(np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
                        global_positions[1:] - global_positions[:-1])
    local_vel = local_vel.reshape(len(local_vel), -1)

    data = root_data
    data = np.concatenate([data, ric_data[:-1]], axis=-1)
    data = np.concatenate([data, rot_data[:-1]], axis=-1)
    #     print(data.shape, local_vel.shape)
    data = np.concatenate([data, local_vel], axis=-1)
    data = np.concatenate([data, feet_l, feet_r], axis=-1)

    return data, global_positions, positions


In [27]:
# Recover global angle and positions for rotation data
# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def recover_root_rot_pos(data):
    rot_vel = data[..., 0]
    r_rot_ang = torch.zeros_like(rot_vel).to(data.device)
    '''Get Y-axis rotation from rotation velocity'''
    r_rot_ang[..., 1:] = rot_vel[..., :-1]
    r_rot_ang = torch.cumsum(r_rot_ang, dim=-1)

    r_rot_quat = torch.zeros(data.shape[:-1] + (4,)).to(data.device)
    r_rot_quat[..., 0] = torch.cos(r_rot_ang)
    r_rot_quat[..., 2] = torch.sin(r_rot_ang)

    r_pos = torch.zeros(data.shape[:-1] + (3,)).to(data.device)
    r_pos[..., 1:, [0, 2]] = data[..., :-1, 1:3]
    '''Add Y-axis rotation to root position'''
    r_pos = qrot(qinv(r_rot_quat), r_pos)

    r_pos = torch.cumsum(r_pos, dim=-2)

    r_pos[..., 1] = data[..., 3]
    return r_rot_quat, r_pos


def recover_from_rot(data, joints_num, skeleton):
    r_rot_quat, r_pos = recover_root_rot_pos(data)

    r_rot_cont6d = quaternion_to_cont6d(r_rot_quat)

    start_indx = 1 + 2 + 1 + (joints_num - 1) * 3
    end_indx = start_indx + (joints_num - 1) * 6
    cont6d_params = data[..., start_indx:end_indx]
    #     print(r_rot_cont6d.shape, cont6d_params.shape, r_pos.shape)
    cont6d_params = torch.cat([r_rot_cont6d, cont6d_params], dim=-1)
    cont6d_params = cont6d_params.view(-1, joints_num, 6)

    positions = skeleton.forward_kinematics_cont6d(cont6d_params, r_pos)

    return positions


def recover_from_ric(data, joints_num):
    r_rot_quat, r_pos = recover_root_rot_pos(data)
    positions = data[..., 4:(joints_num - 1) * 3 + 4]
    positions = positions.view(positions.shape[:-1] + (-1, 3))

    '''Add Y-axis rotation to local joints'''
    positions = qrot(qinv(r_rot_quat[..., None, :]).expand(positions.shape[:-1] + (4,)), positions)

    '''Add root XZ to joints'''
    positions[..., 0] += r_pos[..., 0:1]
    positions[..., 2] += r_pos[..., 2:3]

    '''Concate root and joints'''
    positions = torch.cat([r_pos.unsqueeze(-2), positions], dim=-2)

    return positions

In [30]:
# The given data is used to double check if you are on the right track.
import pickle
input_dir_angle='dataset/inter-human/motions/'
input_dir_pos='dataset/inter-human/new_joints/'
output_dir='dataset/inter-human/new_joint_vecs/'
os.makedirs(output_dir,exist_ok=True)

ex_fps = 20

angle=pickle.load(open(input_dir_angle+"123.pkl",'rb'))
fps = angle['mocap_framerate']
down_sample = round(fps / ex_fps)
bdata=angle["person1"]
angles=np.concatenate((bdata['root_orient'],bdata['pose_body']),axis=1)[::down_sample].reshape(-1,22,3)

positions=pickle.load(open(input_dir_pos+"123.pkl",'rb'))["person1"]
print(positions.shape)
vector=process_file(positions,angles,0.002)[0]
print(vector.dtype)

(23, 22, 3)
float32


In [31]:
for path in tqdm(os.listdir(input_dir_pos)):
    try:
        angle=pickle.load(open(input_dir_angle+path,'rb'))
        fps = angle['mocap_framerate']
        down_sample = round(fps / ex_fps)
        positions=pickle.load(open(input_dir_pos+path,'rb'))
        out={}
        for person in ["person1","person2"]:
            bdata=angle[person]
            rotations=np.concatenate((bdata['root_orient'],bdata['pose_body']),axis=1)[::down_sample].reshape(-1,22,3)
            out[person]=process_file(positions[person],rotations,0.002)[0]
        with open(output_dir+path,'wb') as f:
            pickle.dump(out,f)
    except:
        print(path)

100%|██████████████████████████████████████| 7227/7227 [00:19<00:00, 377.22it/s]


In [ ]:
ve=pickle.load(open(output_dir+"123.pkl",'rb'))
print(pe